<a href="https://colab.research.google.com/github/lydsleepy/machine-learning/blob/main/I310D_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# DATA CLEANING AND SANITIZATION

In [ ]:
# EXPLORATORY DATA ANALYSIS

In [ ]:
# MODEL TRAINING AND FEATURE ENGINEERING
# figuring out which features affect future customer churn the most - experiment with different features & models
# model types we should use: logistic regression, mlpclassifier

In [ ]:
# MODEL EVALUATION AND TESTING

In [ ]:
# DATA VISUALIZATION

In [ ]:
# BIAS AND FAIRNESS EVALUATION